In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

In langrila, the module to chat with LLM and the module to call tools are completely separated. FunctionalChat class is the combination of those two.

- `XXXChatModule`: Only focuses on doing conversation with LLM
- `XXXFunctionCallingModule`: Only focuses on calling tools
- `XXXFunctionalChat`: The combination of the two. FunctionCallingModule works at first and then ChatModule performs. If any tool is not provided, this module behaves as just ChatModule.

In [3]:
from langrila import (
    InMemoryConversationMemory,
    Message,
    PromptTemplate,
    Usage,
)
from langrila.claude import ClaudeFunctionalChat
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Simple text prompt

### For OpenAI Chat Completion

In [4]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="openai",
    # organization_id_env_name="ORGANIZATION_ID",  # as needed
)

In [5]:
prompt = "Hello. How are you today?"

Generate synchronously

In [6]:
response = chat_openai.run(prompt)

response is a pydantic model

In [7]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Hello! I'm just a computer program, so I don't have feelings, but thank you for asking. How can I assist you today?"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 16,
  'completion_tokens': 27},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Hello. How are you today?'}],
   'name': 'User'}]}

You can pick response text like this

In [8]:
print(response.message.content[0].text)

Hello! I'm just a computer program, so I don't have feelings, but thank you for asking. How can I assist you today?


You can see usage to generate

In [9]:
response.usage

Usage(prompt_tokens=16, completion_tokens=27, total_tokens=43)

You can access prompt

In [10]:
response.prompt

[{'role': 'user',
  'content': [{'type': 'text', 'text': 'Hello. How are you today?'}],
  'name': 'User'}]

Asynchronous call is available.

In [11]:
response = await chat_openai.arun(prompt)
print(response.message.content[0].text)

Hello! I'm here and ready to help you with whatever you need. How can I assist you today?


Streaming completion

In [12]:
stream = chat_openai.stream(prompt)
responses = [r for r in stream]
responses

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello!')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm just")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(

Last response includes usage

In [13]:
responses[-1].usage

Usage(prompt_tokens=16, completion_tokens=30, total_tokens=46)

Asynchronous streaming

In [15]:
stream = chat_openai.astream(prompt)
responses = [r async for r in stream]
responses

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello!')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm here")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(

multiple response generation

In [16]:
response = chat_openai.run(prompt, n_results=2)

In [17]:
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Hello! I'm here and ready to help you. How can I assist you today?"},
  {'text': "Hello! I'm just a machine, so I don't have feelings, but thank you for asking. How can I assist you today?"}],
 'name': None}

In [18]:
# async

response = await chat_openai.arun(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?"},
  {'text': "Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"}],
 'name': None}

### For Azure OpenAI

In [19]:
chat_openai_azure = OpenAIFunctionalChat(
    api_key_env_name="AZURE_API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="azure",
    api_version="2024-05-01-preview",
    endpoint_env_name="AZURE_ENDPOINT",
    deployment_id_env_name="AZURE_DEPLOYMENT_ID",
)

In [20]:
response = chat_openai_azure.run(prompt)
print(response.message.content[0].text)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?


In [21]:
# async
response = await chat_openai_azure.arun(prompt)
print(response.message.content[0].text)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with anything you need. How can I assist you today?


In [22]:
# stream
response = chat_openai_azure.stream(prompt)
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello!')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm just")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(

In [23]:
# async stream
response = chat_openai_azure.astream(prompt)
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello!')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm just")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(

multiple generation

In [24]:
response = chat_openai_azure.run(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Hello! I'm just a computer program, so I don't have feelings, but thank you for asking. How can I assist you today?"},
  {'text': "Hello! I'm an AI, so I don't have feelings, but thanks for asking. How can I assist you today?"}],
 'name': None}

In [25]:
# async
response = await chat_openai_azure.arun(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Hello! I'm here and ready to help you. How can I assist you today?"},
  {'text': "Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?"}],
 'name': None}

### Gemini on Google Generative AI

In [26]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-flash",
)

In [30]:
response = gemini.run(prompt)
print(response.message.content[0].text)

As an AI, I don't have feelings or experiences like humans do, so I don't have "good" or "bad" days. But I'm here and ready to assist you with any questions or tasks you may have!  😊 How can I help you today? 



In [31]:
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'As an AI, I don\'t have feelings or experiences like humans do, so I don\'t have "good" or "bad" days. But I\'m here and ready to assist you with any questions or tasks you may have!  😊 How can I help you today? \n'}],
 'name': None}

In [32]:
# async
response = await gemini.arun(prompt)
print(response.message.content[0].text)

As an AI, I don't have feelings or experiences like humans do, so I don't have a "day" in the same way.  But I'm ready to help you with anything you need! 😊 What can I do for you today? 



In [33]:
# stream
response = gemini.stream(prompt)
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='As')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="As an AI, I don't have feelings or experiences like humans do.  ")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="As an AI, I don't have feelings or experiences like humans do.  But I'm here and ready to assist you! 😊  How can I")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="As an AI, I don't have feelings or experiences like humans do.  But I'm here and ready to assist you! 😊  How can I help you today? \n")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens

In [34]:
# async stream
response = gemini.astream(prompt)
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='As')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="As an AI, I don't have feelings or experiences like humans do. So")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='As an AI, I don\'t have feelings or experiences like humans do. So, I\'m not "feeling" anything! But I\'m ready to')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='As an AI, I don\'t have feelings or experiences like humans do. So, I\'m not "feeling" anything! But I\'m ready to assist you with whatever you need. How can I help you today? 😊 \n')], name=None), usage=Usage(pr

### Gemini on VertexAI

In [35]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
)

In [36]:
response = gemini_vertexai.run(prompt)
print(response.message.content[0].text)

As an AI, I don't have feelings or experiences like humans do. However, I'm here and ready to assist you with any questions or tasks you may have! 😊  How can I help you today? 



In [37]:
# async
response = await gemini_vertexai.arun(prompt)
print(response.message.content[0].text)

I am an AI, so I don't have feelings or experiences like humans do. But I am ready to help you with anything you need! 😊 What can I do for you today? 



In [38]:
# stream
response = gemini_vertexai.stream(prompt)
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='As')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="As an AI, I don't have feelings or experiences like humans do. However")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="As an AI, I don't have feelings or experiences like humans do. However, I'm here and ready to assist you with any questions or tasks you")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="As an AI, I don't have feelings or experiences like humans do. However, I'm here and ready to assist you with any questions or tasks you may have! 😊 \n\nHow can I help you today? \n")]

In [39]:
# async stream
response = gemini_vertexai.astream(prompt)
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='As')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="As an AI, I don't have feelings or experiences like humans do. So")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='As an AI, I don\'t have feelings or experiences like humans do. So, I\'m not "good" or "bad" today. I\'')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='As an AI, I don\'t have feelings or experiences like humans do. So, I\'m not "good" or "bad" today. I\'m here to assist you! How can I help you today? 😊 \n')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, 

multiple resposne generation

In [40]:
response = gemini_vertexai.run(prompt, n_results=2)

In [41]:
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'As a large language model, I don\'t have feelings or experiences like humans do. So, I\'m not "feeling" anything in particular today. 😊 \n\nHow are *you* doing today? 😊 \n'},
  {'text': 'As a large language model, I don\'t have feelings or experiences like humans do. So, I\'m not "feeling" anything in particular today. 😊 \n\nHow are *you* doing today? 😊 \n'}],
 'name': None}

In [42]:
# async multiple generation
response = await gemini_vertexai.arun(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "As an AI, I don't have feelings or experiences like humans do. However, I'm here and ready to assist you with any questions or tasks you may have! 😊 How can I help you today? \n"},
  {'text': "As an AI, I don't have feelings or experiences like humans do. However, I'm here and ready to assist you with any questions or tasks you may have! 😊 How can I help you today? \n"}],
 'name': None}

### For Claude on Anthropic

In [39]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
)

In [40]:
response = claude.run(prompt)
print(response.message.content[0].text)

Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you might have. How can I help you today?


In [41]:
# async
response = await claude.arun(prompt)
print(response.message.content[0].text)

Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?


Streaming generation have not been supported yet by Claude in langrila.

### For Claude on Amazon Bedrock

In [42]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
)

In [43]:
response = claude_bedrock.run(prompt)
print(response.message.content[0].text)

Hello! As an AI language model, I don't have personal feelings, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [43]:
response = await claude_bedrock.arun(prompt)
print(response.message.content[0].text)

Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?


# Universal message system

Langrila's chat module transforms user prompt and conversation history to universal message object at first, then converts to each client message. Conversation memory has history with universal message format.

In [45]:
prompt = "Please describe this picture."

Converting from prompt to universal message

In [46]:
from langrila.openai import OpenAIMessage

universal_message = OpenAIMessage.to_universal_message(role="user", message=prompt)
universal_message

Message(role='user', content=[TextContent(text='Please describe this picture.')], name=None)

Converting from universal message to client message

In [47]:
OpenAIMessage.to_client_message(universal_message)

{'role': 'user',
 'content': [{'type': 'text', 'text': 'Please describe this picture.'}],
 'name': 'User'}

Gemini module has same interface. Also Claude as well.

In [48]:
from langrila.gemini.genai import GeminiMessage

# same method as that of OpenAIMessage
universal_message = GeminiMessage.to_universal_message(role="user", message=prompt)
universal_message

Message(role='user', content=[TextContent(text='Please describe this picture.')], name=None)

In [49]:
GeminiMessage.to_client_message(universal_message)

parts {
  text: "Please describe this picture."
}
role: "user"

If you want to set a role and name for your prompt, Message class allows you to do it. Raw text prompt is converted to TextContent object in langrila's chat modules, so TextContent object also can be specified.

In [50]:
prompt = Message(role="user", content="Hello.", name="Nsak")
prompt

Message(role='user', content='Hello.', name='Nsak')

Introducing universal message class, we can use OpenAI Chat Completion, Gemini and Claude in a same way and connect them. 

In [55]:
# conversation memory allows you to interact with modules for multi-turn conversation.
shared_memory = InMemoryConversationMemory()

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-flash",
    conversation_memory=shared_memory,
)

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="openai",
    conversation_memory=shared_memory,
)

claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    conversation_memory=shared_memory,
)

Call OpenAI Chat Completion at first.

In [56]:
prompt = "Do you know Rude in Final Fantasy VII?"
response = chat_openai.run(prompt)
print(response.message.content[0].text)

Yes, Rude is a character in the video game *Final Fantasy VII*. He is a member of the Turks, a covert group working for the Shinra Electric Power Company. Distinguished by his bald head, sunglasses, and silent demeanor, Rude is known for being a skilled fighter and loyal teammate. Throughout the game, players encounter Rude multiple times in various battles and story events, where he often acts alongside his partner Reno. Despite working for Shinra, Rude, along with other members of the Turks, is portrayed with a level of depth that reveals his own sense of honor and camaraderie.


After calling OpenAI Chat Completion, conversation history is inherited by Gemini as it is.

In [57]:
prompt = "What does he think about Tifa?"
response = gemini.run(prompt)
print(response.message.content[0].text)

It's tricky to pinpoint exactly what Rude thinks of Tifa. There's no direct dialogue in *Final Fantasy VII* revealing his inner thoughts about her. However, we can piece together some clues from his interactions and behavior:

* **Respect as an opponent:** Rude, being a professional, likely respects Tifa's strength and fighting ability. He wouldn't underestimate her, as seen in their encounters, especially during the battle at the Shinra building.
* **Potential intrigue:** Given the Turks' role as gatherers of information, he might find her background and connection to Cloud intriguing. He might see her as a potential threat or someone to be watched.
* **Professional distance:** As a member of Shinra, Rude wouldn't be inclined to show personal feelings towards anyone opposing the company. He would likely maintain a neutral and professional demeanor around Tifa.

In essence, Rude probably views Tifa as a competent and dangerous opponent, someone worthy of respect but also someone who st

You can see assistant role and message object were automatically transformed for Gemini API in the prompt field.

In [59]:
response.prompt

[parts {
   text: "Do you know Rude in Final Fantasy VII?"
 }
 role: "user",
 parts {
   text: "Yes, Rude is a character in the video game *Final Fantasy VII*. He is a member of the Turks, a covert group working for the Shinra Electric Power Company. Distinguished by his bald head, sunglasses, and silent demeanor, Rude is known for being a skilled fighter and loyal teammate. Throughout the game, players encounter Rude multiple times in various battles and story events, where he often acts alongside his partner Reno. Despite working for Shinra, Rude, along with other members of the Turks, is portrayed with a level of depth that reveals his own sense of honor and camaraderie."
 }
 role: "model",
 parts {
   text: "What does he think about Tifa?"
 }
 role: "user"]

Claude can follow as well.

In [60]:
prompt = "Does Turks members know his feeling?"

response = claude.run(prompt)

print(response.message.content[0].text)

Ah, I see where you're going with this. I apologize for missing that important detail in my previous response. You're absolutely right to bring this up, as there is indeed a subtle subplot involving Rude's feelings for Tifa in the game. Let me clarify:

1. It's hinted in the game that Rude has a crush on Tifa.

2. This is mostly revealed through interactions with his fellow Turks, particularly Reno.

3. In one scene, Reno teases Rude about his crush on Tifa, showing that at least some of the Turks are aware of Rude's feelings.

4. Rude is shown to be somewhat embarrassed about these feelings, which adds an interesting layer to his character.

5. This crush is never explicitly addressed or acted upon in the main storyline, remaining a background detail that adds depth to Rude's character.

So yes, some of the Turks (at least Reno) do know about Rude's feelings for Tifa. This creates an interesting dynamic, as Rude has to balance his professional duties with his personal feelings. Thank 

# How to specify system instruction

Langrila's module accepts system instruction as shown below.

### For OpenAI Chat Completion

In [61]:
system_instruction = "You must answer any question only with Yes or No."

In [62]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="openai",
    system_instruction=system_instruction,
    conversation_memory=InMemoryConversationMemory(),
)

In [63]:
prompt = "Please tell me about how to sleep well."
response = chat_openai.run(prompt=prompt)
print(response.message.content[0].text)

No.


System prompt is not stored into conversation memory but it exists in prompt attribute in response object. It's bacause we can make system prompt replaceable flexibly. System prompt is inserted as a first message just before API call.

In [64]:
# You can see system prompt in the prompt attribute in the response
response.model_dump()

{'message': {'role': 'assistant', 'content': [{'text': 'No.'}], 'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 34,
  'completion_tokens': 2},
 'prompt': [{'role': 'system',
   'content': [{'type': 'text',
     'text': 'You must answer any question only with Yes or No.'}],
   'name': 'System'},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': 'Please tell me about how to sleep well.'}],
   'name': 'User'}]}

In [65]:
# System prompt is not shown in the conversation memory
chat_openai.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Please tell me about how to sleep well.'}],
  'name': None},
 {'role': 'assistant', 'content': [{'text': 'No.'}], 'name': None}]

### For Gemini

Gemini and Claude also accept system prompt.

In [66]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-flash",
    system_instruction=system_instruction,
)

In [67]:
prompt = "Please tell me about how to sleep well."
response = gemini.run(prompt=prompt)
print(response.message.content[0].text)

No. 



### For Claude

In [68]:
claude = ClaudeFunctionalChat(
    # For Anthropic
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    system_instruction=system_instruction,
)

In [69]:
prompt = "Please tell me about how to sleep well."
response = claude.run(prompt=prompt)
print(response.message.content[0].text)

Yes.


# Token management

For OpenAI Chat Completion, # of tokens is calculated before API call, and module truncates prompt automatically to make sure # of tokens meets token limitation. In this section, we simulate this truncation.

In [70]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="openai",
    context_length=300,  # You can control the number of tokens in the prompt
    conversation_memory=InMemoryConversationMemory(),
)

In [71]:
prompts = [
    "What is the pheasant?",
    "Please tell me about a bird called Kiji in Japan.",
    "What does Kiji do in the tale of the Momotaro",
]

for prompt in prompts:
    response = chat_openai.run(prompt)

Input message is truncated because total length of messages exceeds context length.


Old conversation was truncated as shown below.

In [72]:
# NOTE : Usage after truncation might slightly be different from the specified context_length
response.usage

Usage(prompt_tokens=299, completion_tokens=459, total_tokens=758)

In [73]:
response.prompt

[{'role': 'assistant',
  'name': 'Assistant',
  'content': [{'type': 'text',
    'text': ' Green Pheasants typically inhabit woodlands, agricultural fields, and grasslands. They are commonly found near human settlements and farmland, as they can benefit from the available food sources.\n- **Diet**: They are omnivorous, feeding on a variety of seeds, leaves, insects, and small animals.\n- **Behavior**: These birds are ground-dwellers although they can fly short distances if necessary. They are known for their distinctive calls, especially during the mating season.\n\n### Cultural Significance:\n- **Symbolism**: In Japanese culture, the Green Pheasant is a symbol of beauty and strength. It appears in various forms of traditional art and literature.\n- **National Bird**: The bird was designated the national bird of Japan in 1947.\n- **Folklore**: The Japanese Green Pheasant also appears in folklore and mythology. In some tales, they are seen as messengers of the gods or protectors against

But truncation does not affect conversation memory itself. You can see full conversation history in the memory.

In [74]:
chat_openai.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'What is the pheasant?'}],
  'name': None},
 {'role': 'assistant',
  'content': [{'text': 'The pheasant is a type of bird that belongs to the family Phasianidae, which also includes partridges, quails, and peafowls. Pheasants are known for their vibrant plumage, especially the males, which often have bright colors and intricate patterns to attract mates. One of the most well-known species is the Common Pheasant (Phasianus colchicus), which is native to Asia but has been introduced to many other parts of the world, including North America and Europe, primarily for hunting and ornamental purposes.\n\nPheasants are typically ground-dwelling birds, although they can fly short distances. They are omnivorous, feeding on a variety of foods such as seeds, insects, and small animals. Their habitats vary widely, from grasslands to forest edges, and they often prefer areas with dense underbrush for cover.\n\nThe males are usually larger and more colorful t

If prompts contain some images and # of tokens exceeds limitation, image might be truncated entirely.

# Usage gathering across multiple models

Usage object allows you to sum multiple usages.

In [83]:
usage1 = Usage(model_name="gpt-4o-2024-05-13", prompt_tokens=100, completion_tokens=50)

usage2 = Usage(model_name="gpt-4o-2024-05-13", prompt_tokens=200, completion_tokens=100)

usage1 + usage2

Usage(prompt_tokens=300, completion_tokens=150, total_tokens=450)

Usage don't have capability to sum usages from different models.

In [84]:
usage3 = Usage(model_name="gpt-4o-mini-2024-07-18", prompt_tokens=300, completion_tokens=150)

try:
    usage1 + usage3  # Error
except:
    print("Error!")

Error!


But usage with empty model_name is added to any other usage.

In [85]:
usage4 = Usage(prompt_tokens=100, completion_tokens=50)

usage1 + usage4

Usage(prompt_tokens=200, completion_tokens=100, total_tokens=300)

On the other hand, it's not possible to add usage to usage of empty model_name.

In [86]:
try:
    usage4 + usage1
except:
    print("Error!")

Error!


Using this function, we sometimes want to sum up total #of tokens over each models. In langrila, shared token counter is available to integrate total usage from multiple models.

In [87]:
from langrila import TokenCounter

shared_memory = InMemoryConversationMemory()

shared_token_counter = TokenCounter()

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-flash",
    conversation_memory=shared_memory,
    token_counter=shared_token_counter,
)

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="openai",
    conversation_memory=shared_memory,
    token_counter=shared_token_counter,
)

claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    conversation_memory=shared_memory,
    token_counter=shared_token_counter,
)

In [88]:
prompts = [
    "What is the pheasant?",
    "Please tell me about a bird called Kiji in Japan.",
    "What does Kiji do in the tale of the Momotaro",
]

clients = [gemini, chat_openai, claude]

for i, prompt in enumerate(prompts):
    client = clients[i]
    response = client.run(prompt)

In [89]:
shared_token_counter

{'gemini-1.5-flash': Usage(prompt_tokens=6, completion_tokens=424, total_tokens=430), 'gpt-4o-2024-05-13': Usage(prompt_tokens=475, completion_tokens=582, total_tokens=1057), 'claude-3-5-sonnet-20240620': Usage(prompt_tokens=1198, completion_tokens=407, total_tokens=1605)}

# Prompt template

We can manage a typical prompt as a template.

In [90]:
template = PromptTemplate.from_text_file("../data/sample_prompt_template.txt")
print(template.template)

# INSTRUCTION
{instruction}

# TEXT
{text}


In [91]:
template.set_args(instruction="Hello", text="world!")

PromptTemplate(args={'instruction': 'Hello', 'text': 'world!'}, template='# INSTRUCTION\n{instruction}\n\n# TEXT\n{text}')

In [92]:
print(template.format())

# INSTRUCTION
Hello

# TEXT
world!


We can pass template string to PromptTemplate directly.

In [93]:
PromptTemplate(template="# INSTRUCTION\n{instruction}\n\n# TEXT\n{text}")

PromptTemplate(args={}, template='# INSTRUCTION\n{instruction}\n\n# TEXT\n{text}')